##### Code By Gaurav B R
##### Email: gauravhsn8@gmail.com

# Run the below 4 codes to install the required modules.

In [ ]:
pip install google-api-python-client

In [ ]:
pip install oauth2client

In [ ]:
pip install pandas

In [ ]:
pip install requests

# Metrics Data Extraction (Date wise)
### All metrics will be downloaded in one dataset.
- VIEW_ID:- Add your Universal Analytics view id/property id.
- startDate:- Add the start date in 'YYYY-MM-DD' format.
- endDate:- Add the end date in 'YYYY-MM-DD' format.
- Add the local file path at last for where you want to save the extracted data. Ex: "C:/Users/Gaurav R/Downloads/Metrics_Data.csv"

### Note:-
- Upload the "MetricsDimensions.csv" in your working environment before you run the below programs. Do not change the name of the above file. You can find this file here:-
- Upload the json key file in your working environment and add that file name inside the string for the variable "KEY_FILE_LOCATION".
- Refer this link to get your json key file:- https://bit.ly/43nS9jH

In [ ]:
"""Hello Analytics Reporting API V4."""

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import requests

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = '<uploaded_json_key_file_name>'
VIEW_ID = '123456789'
startDate = 'YYYY-MM-DD'
endDate = 'YYYY-MM-DD'
Metrics = pd.read_csv('MetricsDimensions.csv')['Metrics']
Dimensions = pd.read_csv('MetricsDimensions.csv')['Dimensions']

def initialize_analyticsreporting():
  """Initializes an Analytics Reporting API V4 service object.

  Returns:
    An authorized Analytics Reporting API V4 service object.
  """
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics

def get_report(analytics, page_token=None):
    """Queries the Analytics Reporting API V4.

    Args:
        analytics: An authorized Analytics Reporting API V4 service object.
        page_token: Token for retrieving the next page of data (optional).
    Returns:
        The Analytics Reporting API V4 response.
    """
    report_request = {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': startDate, 'endDate': endDate}],
        'metrics': [{'expression': 'ga:users'}],
        'dimensions': [{'name': 'ga:pagePath'}, {'name': 'ga:date'}],
        'pageSize': 1000000
    }

    if page_token:
        report_request['pageToken'] = page_token

    return analytics.reports().batchGet(
        body={
            'reportRequests': [report_request]
        }
    ).execute()


def process_response(response):
    """Processes the Analytics Reporting API V4 response and returns a Pandas DataFrame.

    Args:
        response: An Analytics Reporting API V4 response.

    Returns:
        A Pandas DataFrame containing the processed data.
    """
    data = []
    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

        for row in report.get('data', {}).get('rows', []):
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            row_data = []
            for header, dimension in zip(dimensionHeaders, dimensions):
                row_data.append(dimension)

            for i, values in enumerate(dateRangeValues):
                for metricHeader, value in zip(metricHeaders, values.get('values')):
                    row_data.append(value)

            data.append(row_data)

    # Create a Pandas DataFrame
    df = pd.DataFrame(data, columns=dimensionHeaders + [header['name'] for header in metricHeaders])
    return df


analytics = initialize_analyticsreporting()
response = get_report(analytics)
df = process_response(response)

# Check if there are more pages
next_page_token = response.get('reports', [{}])[0].get('nextPageToken')
while next_page_token:
  response = get_report(analytics, next_page_token)
  df1 = process_response(response)
  next_page_token = response.get('reports', [{}])[0].get('nextPageToken')
  df=pd.concat([df,df1],axis=0)
df['Merged (ga:pagePath + ga:date)']=df['ga:pagePath']+' '+df['ga:date']


def get_report1(analytics, page_token=None, a='hello'):
    """Queries the Analytics Reporting API V4.

    Args:
        analytics: An authorized Analytics Reporting API V4 service object.
        page_token: Token for retrieving the next page of data (optional).
    Returns:
        The Analytics Reporting API V4 response.
    """
    report_request = {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': startDate, 'endDate': endDate}],
        'metrics': [{'expression': 'ga:users'}, {'expression': a}],
        'dimensions': [{'name': 'ga:pagePath'}, {'name': 'ga:date'}],
        'pageSize': 100000
    }

    if page_token:
        report_request['pageToken'] = page_token

    return analytics.reports().batchGet(
        body={
            'reportRequests': [report_request]
        }
    ).execute()


def process_response1(response):
    """Processes the Analytics Reporting API V4 response and returns a Pandas DataFrame.

    Args:
        response: An Analytics Reporting API V4 response.

    Returns:
        A Pandas DataFrame containing the processed data.
    """
    data = []
    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

        for row in report.get('data', {}).get('rows', []):
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            row_data = []
            for header, dimension in zip(dimensionHeaders, dimensions):
                row_data.append(dimension)

            for i, values in enumerate(dateRangeValues):
                for metricHeader, value in zip(metricHeaders, values.get('values')):
                    row_data.append(value)

            data.append(row_data)

    # Create a Pandas DataFrame
    df1 = pd.DataFrame(data, columns=dimensionHeaders + [header['name'] for header in metricHeaders])
    return df1


metrics_error=[]
for i in Metrics:
  try:
    analytics = initialize_analyticsreporting()
    response = get_report1(analytics, a=i)
    df1 = process_response1(response)

    # Check if there are more pages
    next_page_token = response.get('reports', [{}])[0].get('nextPageToken')
    while next_page_token:
      response = get_report1(analytics, next_page_token, a=i)
      df2 = process_response1(response)
      next_page_token = response.get('reports', [{}])[0].get('nextPageToken')
      df1=pd.concat([df1,df2],axis=0)
    df1['Merged (ga:pagePath + ga:date)']=df1['ga:pagePath']+' '+df1['ga:date']
    df1=df1.drop(columns=['ga:pagePath', 'ga:date', 'ga:users'])
    df=pd.merge(df, df1, on='Merged (ga:pagePath + ga:date)', how='outer')
  except Exception as e:
    metrics_error.extend([i])
    continue
df
df.to_csv("<Add_the_file_path_here>/Metrics_Data.csv") # Add the local file path here where you want to save the extracted data. Ex: "C:/Users/Gaurav R/Downloads/Metrics_Data.csv"

# Dimensions Data Extraction
### Each dimension will be extracted in a seperate dataset and will be saved in the file path which you provide below.
- VIEW_ID:- Add your Universal Analytics view id/property id.
- startDate:- Add the start date in 'YYYY-MM-DD' format.
- endDate:- Add the end date in 'YYYY-MM-DD' format.
- Add the local file path at last for where you want to save the extracted data. Ex: "C:/Users/Gaurav R/Downloads/......."

In [ ]:
"""Hello Analytics Reporting API V4."""

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import requests


SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = '<uploaded_json_key_file_name>'
VIEW_ID = '123456789'
startDate = 'YYYY-MM-DD'
endDate = 'YYYY-MM-DD'
Metrics = pd.read_csv('MetricsDimensions.csv')['Metrics']
Dimensions = pd.read_csv('MetricsDimensions.csv')['Dimensions']

def initialize_analyticsreporting():
  """Initializes an Analytics Reporting API V4 service object.

  Returns:
    An authorized Analytics Reporting API V4 service object.
  """
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics

def process_response(response):
    """Processes the Analytics Reporting API V4 response and returns a Pandas DataFrame.

    Args:
        response: An Analytics Reporting API V4 response.

    Returns:
        A Pandas DataFrame containing the processed data.
    """
    data = []
    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

        for row in report.get('data', {}).get('rows', []):
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            row_data = []
            for header, dimension in zip(dimensionHeaders, dimensions):
                row_data.append(dimension)

            for i, values in enumerate(dateRangeValues):
                for metricHeader, value in zip(metricHeaders, values.get('values')):
                    row_data.append(value)

            data.append(row_data)

    # Create a Pandas DataFrame
    df = pd.DataFrame(data, columns=dimensionHeaders + [header['name'] for header in metricHeaders])
    return df

def get_report(analytics, page_token=None, a='hello'):
    """Queries the Analytics Reporting API V4.

    Args:
        analytics: An authorized Analytics Reporting API V4 service object.
        page_token: Token for retrieving the next page of data (optional).
    Returns:
        The Analytics Reporting API V4 response.
    """
    report_request = {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': startDate, 'endDate': endDate}],
        'metrics': [{'expression': 'ga:users'}, {'expression': a}],
        'dimensions': [{'name': 'ga:pagePath'}, {'name': 'ga:date'}, {'name': 'ga:dateHourMinute'}],
        'pageSize': 100000
    }

    if page_token:
        report_request['pageToken'] = page_token

    return analytics.reports().batchGet(
        body={
            'reportRequests': [report_request]
        }
    ).execute()

metrics_error=[]
for i in Metrics:
    try:
      analytics = initialize_analyticsreporting()
      response = get_report(analytics, a=i)
      df = process_response(response)

      # Check if there are more pages
      next_page_token = response.get('reports', [{}])[0].get('nextPageToken')
      while next_page_token:
        response = get_report(analytics, next_page_token, a=i)
        df1 = process_response(response)
        next_page_token = response.get('reports', [{}])[0].get('nextPageToken')
        df=pd.concat([df,df1],axis=0)
      df['Merged (ga:pagePath + ga:date)']=df['ga:pagePath']+' '+df['ga:date']
      df.to_csv(f"<Add_the_folder_path_here>/{i[3:]}.csv") # Add the local file path here where you want to save the extracted data. Ex: "C:/Users/Gaurav R/Downloads/{i[3:]}.csv"
    except Exception as e:
      metrics_error.extend([i])
      continue

# Metrics Data Extraction (Date_Hour_Minute wise)
### Each metrics will be extracted in a seperate dataset and will be saved in the file path which you provide below.
- VIEW_ID:- Add your Universal Analytics view id/property id.
- startDate:- Add the start date in 'YYYY-MM-DD' format.
- endDate:- Add the end date in 'YYYY-MM-DD' format.
- Add the local file path at last for where you want to save the extracted data. Ex: "C:/Users/Gaurav R/Downloads/......."

In [ ]:
"""Hello Analytics Reporting API V4."""

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import requests


SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = '<uploaded_json_key_file_name>'
VIEW_ID = '123456789'
startDate = 'YYYY-MM-DD'
endDate = 'YYYY-MM-DD'
Metrics = pd.read_csv('MetricsDimensions.csv')['Metrics']
Dimensions = pd.read_csv('MetricsDimensions.csv')['Dimensions']

def initialize_analyticsreporting():
  """Initializes an Analytics Reporting API V4 service object.

  Returns:
    An authorized Analytics Reporting API V4 service object.
  """
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics

def process_response(response):
    """Processes the Analytics Reporting API V4 response and returns a Pandas DataFrame.

    Args:
        response: An Analytics Reporting API V4 response.

    Returns:
        A Pandas DataFrame containing the processed data.
    """
    data = []
    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

        for row in report.get('data', {}).get('rows', []):
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            row_data = []
            for header, dimension in zip(dimensionHeaders, dimensions):
                row_data.append(dimension)

            for i, values in enumerate(dateRangeValues):
                for metricHeader, value in zip(metricHeaders, values.get('values')):
                    row_data.append(value)

            data.append(row_data)

    # Create a Pandas DataFrame
    df = pd.DataFrame(data, columns=dimensionHeaders + [header['name'] for header in metricHeaders])
    return df

def get_report(analytics, page_token=None, a='hello'):
    """Queries the Analytics Reporting API V4.

    Args:
        analytics: An authorized Analytics Reporting API V4 service object.
        page_token: Token for retrieving the next page of data (optional).
    Returns:
        The Analytics Reporting API V4 response.
    """
    report_request = {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': startDate, 'endDate': endDate}],
        'metrics': [{'expression': 'ga:users'}, {'expression': a}],
        'dimensions': [{'name': 'ga:pagePath'}, {'name': 'ga:date'}, {'name': 'ga:dateHourMinute'}],
        'pageSize': 100000
    }

    if page_token:
        report_request['pageToken'] = page_token

    return analytics.reports().batchGet(
        body={
            'reportRequests': [report_request]
        }
    ).execute()

metrics_error=[]
for i in Metrics:
    try:
      analytics = initialize_analyticsreporting()
      response = get_report(analytics, a=i)
      df = process_response(response)

      # Check if there are more pages
      next_page_token = response.get('reports', [{}])[0].get('nextPageToken')
      while next_page_token:
        response = get_report(analytics, next_page_token, a=i)
        df1 = process_response(response)
        next_page_token = response.get('reports', [{}])[0].get('nextPageToken')
        df=pd.concat([df,df1],axis=0)
      df['Merged (ga:pagePath + ga:date)']=df['ga:pagePath']+' '+df['ga:date']
      df.to_csv(f"<Add_the_folder_path_here>/{i[3:]}.csv") # Add the local file path at last for where you want to save the extracted data. Ex: "C:/Users/Gaurav R/Downloads/{i[3:]}.csv"
    except Exception as e:
      metrics_error.extend([i])
      continue